In [ ]:
this will get the fraction of burned area per 4km grid cell in each year for FireCII

In [10]:
import os
import numpy as np
import rasterio as rio
from rasterio.warp import reproject, Resampling
from tqdm import tqdm

# ---------------- USER PATHS ----------------
TEMPLATE = "/explore/nobackup/people/spotter5/clelland_fire_ml/tem_grid.tif"
IN_DIR   = "/explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us"
OUT_DIR  = "/explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac"

os.makedirs(OUT_DIR, exist_ok=True)
YEARS = range(2001, 2020)
YEARS = [i for i in range(2001, 2020) if i != 2004]

# ------------- LOAD TEMPLATE GRID SPECS -------------
with rio.open(TEMPLATE) as tds:
    tmpl_crs = tds.crs
    tmpl_transform = tds.transform
    tmpl_height = tds.height
    tmpl_width  = tds.width
    tmpl_bounds = tds.bounds

print("Template:")
print("  CRS:", tmpl_crs)
print("  size (W,H):", tmpl_width, tmpl_height)
print("  transform:", tmpl_transform)
print("  bounds:", tmpl_bounds)

def process_one(year):
    src_path = os.path.join(IN_DIR, f"{year}.tif")
    dst_path = os.path.join(OUT_DIR, f"{year}.tif")

    if not os.path.exists(src_path):
        print(f"[MISS] {src_path}")
        return
    if os.path.exists(dst_path):
        print(f"[SKIP] {dst_path}")
        return

    with rio.open(src_path) as src:
        src_crs = src.crs
        src_transform = src.transform
        src_nodata = src.nodata  # may be None

        # read first band (assumes single-band annual FireCCI)
        arr = src.read(1)  # shape (H, W)

        # Build uint8 mask: 1 if >0, 0 if <=0; nodata/NaN -> 255 to ignore in average
        is_nan = np.isnan(arr)
        if src_nodata is not None:
            is_nodata = is_nan | (arr == src_nodata)
        else:
            is_nodata = is_nan

        mask = np.zeros(arr.shape, dtype=np.uint8)
        mask[(arr > 0) & (~is_nodata)] = 1
        mask[is_nodata] = 255  # src_nodata for averaging

        # Reproject to template using average; destination is float32 fraction 0..1
        dst = np.full((tmpl_height, tmpl_width), np.nan, dtype=np.float32)

        reproject(
            source=mask,
            destination=dst,
            src_transform=src_transform,
            src_crs=src_crs,
            src_nodata=255,                 # ignore our 255 nodata
            dst_transform=tmpl_transform,
            dst_crs=tmpl_crs,
            dst_nodata=np.nan,              # keep NaNs in memory
            resampling=Resampling.average,  # fraction of 1s in each 4 km cell
            num_threads=0,                  # ALL_CPUS
        )

        # Prepare output profile; use a finite nodata (e.g., -9999.0) for GeoTIFF
        out_profile = {
            "driver": "GTiff",
            "height": tmpl_height,
            "width": tmpl_width,
            "count": 1,
            "dtype": "float32",
            "crs": tmpl_crs,
            "transform": tmpl_transform,
            "tiled": True,
            "blockxsize": 256,
            "blockysize": 256,
            "compress": "LZW",
            "BIGTIFF": "IF_SAFER",
            "nodata": -9999.0,
        }

        # Replace NaN with -9999 for writing (so nodata is valid & finite)
        out_data = dst.copy()
        np.nan_to_num(out_data, copy=False, nan=-9999.0)

        with rio.open(dst_path, "w", **out_profile) as dst_ds:
            dst_ds.write(out_data, 1)

    print(f"[OK] {dst_path} (0–1 burned fraction)")

for y in tqdm(YEARS):
    process_one(y)

print("Done.")


Template:
  CRS: EPSG:6931
  size (W,H): 2242 1934
  transform: | 4000.00, 0.00,-4602000.00|
| 0.00,-4000.00, 4251000.00|
| 0.00, 0.00, 1.00|
  bounds: BoundingBox(left=-4602000.0, bottom=-3485000.0, right=4366000.0, top=4251000.0)


  6%|▌         | 1/18 [05:32<1:34:06, 332.17s/it]

[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac/2001.tif (0–1 burned fraction)


 11%|█         | 2/18 [10:39<1:24:43, 317.73s/it]

[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac/2002.tif (0–1 burned fraction)
[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac/2003.tif (0–1 burned fraction)


...

[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac/2017.tif (0–1 burned fraction)


 94%|█████████▍| 17/18 [1:29:28<05:10, 310.45s/it]

[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac/2018.tif (0–1 burned fraction)


100%|██████████| 18/18 [1:33:56<00:00, 313.11s/it]

[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us_temgrid_frac/2019.tif (0–1 burned fraction)
Done.


In [11]:
't'

't'

Snap land cover to tem grid

In [12]:
from osgeo import gdal, osr
import os
from pathlib import Path

# -------- paths (Linux) --------
TEMPLATE = "/explore/nobackup/people/spotter5/clelland_fire_ml/tem_grid.tif"
SRC      = "/explore/nobackup/people/spotter5/clelland_fire_ml/upscaled_tem_landcover_map_g1.tif"
OUT      = "/explore/nobackup/people/spotter5/clelland_fire_ml/upscaled_tem_landcover_map_g1_temgrid.tif"

# GDAL perf tweaks
gdal.UseExceptions()
os.environ.setdefault("GDAL_NUM_THREADS", "ALL_CPUS")
gdal.SetConfigOption("GDAL_CACHEMAX", "2048")  # MB

# --- read template specs ---
tmpl_ds = gdal.Open(TEMPLATE, gdal.GA_ReadOnly)
if tmpl_ds is None:
    raise RuntimeError(f"Cannot open template: {TEMPLATE}")

tmpl_gt   = tmpl_ds.GetGeoTransform()
tmpl_w    = tmpl_ds.RasterXSize
tmpl_h    = tmpl_ds.RasterYSize
tmpl_wkt  = tmpl_ds.GetProjectionRef()
x0, px, rx, y0, ry, py = tmpl_gt
left   = x0
top    = y0
right  = x0 + px * tmpl_w + rx * tmpl_h
bottom = y0 + ry * tmpl_w + py * tmpl_h
xmin, xmax = min(left, right), max(left, right)
ymin, ymax = min(bottom, top), max(bottom, top)
bounds = (xmin, ymin, xmax, ymax)
tmpl_ds = None

# --- warp options (NEAREST for categorical) ---
warp_opts = gdal.WarpOptions(
    format="GTiff",
    outputBounds=bounds,
    width=tmpl_w, height=tmpl_h,
    dstSRS=tmpl_wkt,
    resampleAlg=gdal.GRA_NearestNeighbour,  # categorical snapping
    multithread=True,
    warpOptions=["NUM_THREADS=ALL_CPUS", "INIT_DEST=NO_DATA"],
    # If your source has a known nodata, you can set srcNodata=... here
    dstNodata=None,  # keep as-is
    creationOptions=[
        "COMPRESS=LZW",
        "TILED=YES",
        "BIGTIFF=IF_SAFER",
        "BLOCKXSIZE=256",
        "BLOCKYSIZE=256"
    ]
)

# --- run warp ---
Path(OUT).parent.mkdir(parents=True, exist_ok=True)
out_ds = gdal.Warp(OUT, SRC, options=warp_opts)
if out_ds is None:
    raise RuntimeError("gdal.Warp returned None")
out_ds = None

print(f"[OK] Snapped to template grid:\n  {SRC}\n→ {OUT}")


[OK] Snapped to template grid:
  /explore/nobackup/people/spotter5/clelland_fire_ml/upscaled_tem_landcover_map_g1.tif
→ /explore/nobackup/people/spotter5/clelland_fire_ml/upscaled_tem_landcover_map_g1_temgrid.tif


Now for topographic variables

In [2]:
from osgeo import gdal
import os
from pathlib import Path

# ---- Paths ----
TEMPLATE = "/explore/nobackup/people/spotter5/clelland_fire_ml/tem_grid.tif"
SRC      = "/explore/nobackup/people/spotter5/clelland_fire_ml/glo30_4km_epsg3413_elev_slope_aspectrad_n40.tif"
OUT      = "/explore/nobackup/people/spotter5/clelland_fire_ml/glo30_4km_epsg3413_elev_slope_aspectrad_n40_temgrid.tif"

# GDAL perf tweaks
gdal.UseExceptions()
os.environ.setdefault("GDAL_NUM_THREADS", "ALL_CPUS")
gdal.SetConfigOption("GDAL_CACHEMAX", "2048")  # MB

# --- read template grid specs (CRS, extent, size) ---
tmpl = gdal.Open(TEMPLATE, gdal.GA_ReadOnly)
if tmpl is None:
    raise RuntimeError(f"Cannot open template: {TEMPLATE}")

gt  = tmpl.GetGeoTransform()
W   = tmpl.RasterXSize
H   = tmpl.RasterYSize
wkt = tmpl.GetProjectionRef()
x0, px, rx, y0, ry, py = gt
left   = x0
top    = y0
right  = x0 + px * W + rx * H
bottom = y0 + ry * W + py * H
xmin, xmax = min(left, right), max(left, right)
ymin, ymax = min(bottom, top), max(bottom, top)
bounds = (xmin, ymin, xmax, ymax)
tmpl = None

# --- warp (bilinear for continuous variables) ---
opts = gdal.WarpOptions(
    format="GTiff",
    outputBounds=bounds,
    width=W, height=H,
    dstSRS=wkt,
    resampleAlg=gdal.GRA_Bilinear,
    multithread=True,
    warpOptions=["NUM_THREADS=ALL_CPUS", "INIT_DEST=NO_DATA"],
    # Keep nodata if present; you can set explicit src/dst nodata here if needed, e.g. srcNodata=-9999
    creationOptions=[
        "COMPRESS=LZW",
        "TILED=YES",
        "BIGTIFF=IF_SAFER",
        "BLOCKXSIZE=256",
        "BLOCKYSIZE=256",
    ],
)

Path(OUT).parent.mkdir(parents=True, exist_ok=True)
out_ds = gdal.Warp(OUT, SRC, options=opts)
if out_ds is None:
    raise RuntimeError("gdal.Warp returned None")
out_ds = None

print(f"[OK] Snapped:\n  {SRC}\n→ {OUT}")


[OK] Snapped:
  /explore/nobackup/people/spotter5/clelland_fire_ml/glo30_4km_epsg3413_elev_slope_aspectrad_n40.tif
→ /explore/nobackup/people/spotter5/clelland_fire_ml/glo30_4km_epsg3413_elev_slope_aspectrad_n40_temgrid.tif
